In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-nov-2021/train.csv
/kaggle/input/tabular-playground-series-nov-2021/test.csv


##### Display Setting 

This we set 103 the maximum feature length so that all the features can be vissible by scrolling in the output window 

In [2]:
pd.set_option('display.max_columns',120) # setting the visulaization on columns
pd.set_option('display.max_rows',120)
pd.set_option("display.max_colwidth",100)

# IMPORTING LIBRARIES

In [3]:
import numpy as np # Linear algebra.
import scipy.stats as stat
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv).
import datatable as dt # Data processing, CSV file I/O (e.g. dt.fread).


# Visualization.
import seaborn as sns 
import matplotlib.pyplot as plt 

# Machine Learning 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import make_pipeline

from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')


# LOAD DATA 

In [4]:
df_train =dt.fread('../input/tabular-playground-series-nov-2021/train.csv').to_pandas()
df_test = dt.fread('../input/tabular-playground-series-nov-2021/test.csv').to_pandas()
submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

df_train['target'] = df_train['target'].astype('int32') # Datatable reads target as bool by default.
print(f'{df_train.info()}\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 102 entries, id to target
dtypes: float64(100), int32(2)
memory usage: 462.3 MB
None




In [5]:
df_train.shape,df_test.shape

((600000, 102), (540000, 101))

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 102 entries, id to target
dtypes: float64(100), int32(2)
memory usage: 462.3 MB


### Downcasting the data

Special thanks for  Sergie for some of the excelent ideas to reduce the load by downcasting and sampling. 
https://www.kaggle.com/sergiosaharovskiy/tps-nov-2021-a-complete-guide

In [7]:
memory_train = sum(df_train.memory_usage()) / 1e6
print(f'[INFO] Memory usage train_before: {memory_train:.2f} MB.')

memory_test = sum(df_test.memory_usage()) / 1e6
print(f'[INFO] Memory usage test_before: {memory_test:.2f} MB.\n')

# Downcasting the traind dataset.
for col in df_train.columns:
    
    if df_train[col].dtype == "float64":
        df_train[col] = pd.to_numeric(df_train[col], downcast="float")
        
    if df_train[col].dtype == "int64":
        df_train[col] = pd.to_numeric(df_train[col], downcast="integer")
        
# Downcasting the test dataset.
for col in df_test.columns:
    
    if df_test[col].dtype == "float64":
        df_test[col] = pd.to_numeric(df_test[col], downcast="float")
        
    if df_test[col].dtype == "int64":
        df_test[col] = pd.to_numeric(df_test[col], downcast="integer")
        
memory_train = sum(df_train.memory_usage()) / 1e6
print(f'[INFO] Memory usage train: {memory_train:.2f} MB.')

memory_test = sum(df_test.memory_usage()) / 1e6
print(f'[INFO] Memory usage test: {memory_test:.2f} MB.')

[INFO] Memory usage train_before: 484.80 MB.
[INFO] Memory usage test_before: 434.16 MB.

[INFO] Memory usage train: 244.80 MB.
[INFO] Memory usage test: 218.16 MB.


# Data Cleaning and Handling NaN

In [8]:
#df_train.isnull().sum()

In [9]:
#df_test.isnull().sum()

No Null values in Train and Test Data

## Below codes to check the presence of Nan values in each features

In [10]:
# for i in df_train.columns:
#     if df_train[i].isnull().sum() !=0:
#         print(i, df_train[i].isnull().sum())
#     else:
#         print('No Null present in ',i)

In [11]:
# for i in df_test.columns:
#     if df_test[i].isnull().sum() !=0:
#         print(i, df_test[i].isnull().sum())
#     else:
#         print('No Null present in ',i)

## *********

In [12]:
df_train['target'].value_counts(normalize=True) #ehecking the ratio of target class

1    0.50601
0    0.49399
Name: target, dtype: float64

The ratios of binary class is 50 :49.3 % , so this is a well balanced training dataset

In [13]:
df_train.shape,df_test.shape

((600000, 102), (540000, 101))

In [14]:
dataset= pd.concat(objs=[df_train,df_test],axis=0).reset_index(drop=True) #combining both test and train for easy preoprocessing and feature transformation
dataset.shape

(1140000, 102)

### Checking the data distrubution

In [15]:
#dataset.iloc[:,1:].describe().T.sort_values(by='std',ascending=False).head(7)

In [16]:
#dataset.iloc[:,1:].describe().T.sort_values(by='std',ascending=False).tail(5)

Here we checked the standard deviation of the features , the max is 526.32 , and min std is 0.049.
So we have to address the highest std features f2,f35 ,f44

## Plot every features to check whether it is in  Gaussian distribution or Not

In [17]:
# fig, axes = plt.subplots(10,10, figsize=(25, 15))
# axes = axes.flatten()

# for idx, ax in enumerate(axes):
    
#     sns.kdeplot(
#         data=df_train, ax=ax, hue='target', fill=True,
#         x=f'f{idx}', palette=['#4DB6AC', 'red'], legend=idx==0
#     )
    
#     ax.set_title(f'f{idx}', loc='right', weight='bold', fontsize=12)


We can observe that many are non gaussian distributed data,eg: f0, f2 ,f4 etc....
So thees features need feature transformation to bring it into gaussian distrubution or very close to Gaussian distrubution.

There are many festure transoformations like log, square root,exponential, reciprocal and power transformers like Yeo-Johnson and Box-Cox Transformers.

To make things simple we first try log transformation for all the features, then to improve the model we can try different transformers for each features and check the distribution

In [18]:
#features required transformation since not in normal distrubution
non_gaus= ['f0','f2','f4','f9','f12','f16','f19','f20','f23','f24','f27',
    'f28','f30','f31','f32','f33','f35','f39','f42','f44','f46','f48',
    'f49','f51','f52','f53','f56','f58','f59','f60','f61','f62','f63',
    'f64','f68','f69','f72','f73','f75','f76','f78','f79','f81','f83',
    'f84','f87','f88','f89','f90','f92','f93','f94','f95','f98','f99' ]

In [19]:
df_train[non_gaus].head(6)

,f0,f2,f4,f9,f12,f16,f19,f20,f23,f24,f27,f28,f30,f31,f32,f33,f35,f39,f42,f44,f46,f48,f49,f51,f52,f53,f56,f58,f59,f60,f61,f62,f63,f64,f68,f69,f72,f73,f75,f76,f78,f79,f81,f83,f84,f87,f88,f89,f90,f92,f93,f94,f95,f98,f99
0,0.106643,132.804001,0.081971,0.012330,0.560137,0.197527,-3.842450,0.037380,0.009397,0.144738,0.033225,-0.018284,-0.009294,-0.036271,-0.049871,0.019483,11.286300,-0.105772,0.789430,1.491570,0.041824,0.053691,1.701270,0.094902,0.030220,0.597024,-0.068686,0.061044,0.046099,0.017113,-0.027552,0.019483,-0.048825,0.050748,0.063342,0.121043,0.167613,0.039754,-0.016614,0.107679,0.013660,-0.097023,0.244457,0.113090,-0.015472,0.037227,-0.118814,0.067086,0.010739,0.013331,-0.011715,0.052759,0.065400,0.085974,0.240496
1,0.125021,76.533600,0.099400,0.037706,0.062040,0.233586,0.067874,-0.051268,0.103441,0.067638,-0.181834,0.008359,0.011850,0.022292,0.069320,0.117109,24.480700,0.092358,0.005896,2.564530,0.025997,1.575540,0.066105,0.025247,0.183500,0.110038,0.188196,0.136087,-0.008493,-0.015264,-0.010841,0.064584,0.102548,0.093611,0.091289,-0.036360,0.000657,0.051302,0.123294,-0.022671,-0.010397,0.058330,-0.118386,-0.009112,0.178701,0.005244,0.121381,0.109968,0.135838,0.017054,0.124863,0.154064,0.606848,-0.020877,0.024719
2,0.036330,233.546005,0.026914,0.056995,0.105244,-0.262018,0.020542,-0.008806,0.038180,0.123716,0.120601,0.082069,0.002270,0.045182,0.014405,0.011599,33.738201,2.122030,-0.006822,17.345699,-0.033600,0.051978,-0.002004,0.018367,-0.030468,0.111409,-0.019944,0.068105,-0.009812,-0.010627,0.027571,-0.007121,-0.048914,-0.002574,0.057749,0.525174,0.089848,0.093744,-0.010978,0.076671,0.038691,0.382731,-0.121482,0.147098,-0.016565,0.078828,0.979808,0.026758,0.117310,0.085222,0.032396,0.116092,-0.001689,0.124464,0.148209
3,-0.014077,779.966980,0.006948,0.123494,0.058411,0.175475,0.042086,0.005141,0.111813,0.121641,-0.032843,0.058168,0.097465,0.072744,0.000324,0.063362,25.382401,1.085670,0.034631,71.448502,0.092245,0.044829,0.027195,0.046955,0.063695,0.029225,0.164284,0.062306,-0.021406,0.009471,0.110884,0.026837,2.931160,0.068112,0.035611,-0.139274,0.117877,0.065605,-0.058029,0.070501,0.068559,0.162928,-0.008835,0.913739,-0.163204,-0.048894,4.917990,0.069930,-0.015347,-0.017103,-0.008100,0.062013,0.041193,0.040017,0.044873
4,-0.003259,156.128006,0.018284,0.034166,0.057555,0.287104,0.074883,-0.010543,0.008023,0.045235,-0.009376,0.528966,0.020000,0.106828,0.051307,0.045939,15.561500,0.155748,0.072155,3.171160,-0.105861,0.090698,0.112919,0.126866,0.142731,0.055724,0.524051,0.115334,0.125205,0.067432,0.075586,0.032114,-0.042284,0.047973,0.034019,0.024611,0.082462,-0.023296,0.086238,0.157568,0.061247,0.086603,1.411090,-0.023154,0.011234,0.044521,0.375731,0.134351,0.013781,-0.042943,0.105616,0.125072,0.037509,-0.012819,0.072798
5,0.026798,136.598007,0.046621,0.097730,0.138280,0.054934,-0.232102,-0.008823,-0.044461,0.117378,0.191585,0.223433,0.046938,0.074192,0.010365,0.019538,102.540001,0.047628,0.044833,2.362230,0.018796,0.652710,0.068950,0.100434,0.090060,0.130640,1.254380,0.055039,0.082419,0.072128,0.046595,0.183864,0.051966,0.034643,-0.043684,0.075303,0.055129,0.053770,0.042309,0.161307,0.028935,-0.002682,0.193480,0.135856,0.054054,0.040915,1.317220,0.133069,-0.013443,0.101817,0.058697,0.024684,0.102794,0.159111,0.111462


### Log Transformation

In [20]:
df_transform_log = df_train[non_gaus]

value_neg = (df_transform_log  < 0) #created a mask for negative values
value_pos = (df_transform_log  > 0) #created a mask for positive values

df_transform_log[value_neg]=np.log(np.abs(df_transform_log))  * (-1)
df_transform_log[value_pos]=np.log(df_transform_log)

### Data distribution after log transformation

In [21]:
# df_transform_log['target'] = df_train['target']

# fig, axes = plt.subplots(11,5, figsize=(20, 12))
# axes = axes.flatten()

# for col, ax in zip(non_gaus, axes):
    
#     sns.kdeplot(
#         data=df_transform_log, ax=ax, hue='target', fill=True,
#         x=col, palette=['#4DB6AC', 'red'], legend=idx==0
#     )
#     ax.set_title(f'f{idx}', loc='right', weight='bold', fontsize=12)

### Apply log transform to the whole dataset (train and test together)

In [22]:
dataset[non_gaus] # Features from the dataset we need to give log transform

value_neg = (dataset[non_gaus]  < 0)
value_pos = (dataset[non_gaus]  > 0)

dataset[non_gaus][value_neg]=np.log(np.abs(dataset[non_gaus]))  * (-1)
dataset[non_gaus][value_pos]=np.log(dataset[non_gaus])

### Adding aggregate features 

In [23]:
features=dataset.columns[1:-1]
#SIMPLE FEATURE ENGINEERING, CREATING SOME AGGREGATION FEATURES
dataset['sum']=dataset[features].sum(axis=1)

dataset['mean']=dataset[features].mean(axis=1)


dataset['std'] = dataset[features].std(axis=1)

dataset['max'] = dataset[features].max(axis=1)

dataset['min'] = dataset[features].min(axis=1)


dataset['kurt'] = dataset[features].kurtosis(axis=1)

agg_features= ['sum','mean','std','max','min','kurt']

# Model Creation

In [24]:
dataset.head(3)

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,target,sum,mean,std,max,min,kurt
0,0,0.106643,3.59437,132.804001,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,0.012330,1.607190,-0.318058,0.560137,2.80688,1.35114,2.535930,0.197527,0.676494,1.98979,-3.842450,0.037380,0.230322,3.33055,0.009397,0.144738,3.05131,1.30362,0.033225,-0.018284,2.74821,-0.009294,-0.036271,-0.049871,0.019483,3.898460,11.286300,1.13802,3.366880,4.94446,-0.105772,2.11345,3.452230,0.789430,1.113210,1.491570,2.440370,0.041824,3.35537,0.053691,1.701270,0.908831,0.094902,0.030220,0.597024,4.44341,1.586490,-0.068686,-0.108268,0.061044,0.046099,0.017113,-0.027552,0.019483,-0.048825,0.050748,3.729300,5.017440,4.186880,0.063342,0.121043,1.37175,4.017450,0.167613,0.039754,2.04236,-0.016614,0.107679,3.507250,0.013660,-0.097023,5.39607,0.244457,3.49184,0.113090,-0.015472,4.208790,4.10656,0.037227,-0.118814,0.067086,0.010739,1.09862,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0.0,265.660645,2.656606,13.288189,132.804001,-3.842450,95.667130
1,1,0.125021,1.67336,76.533600,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,0.037706,0.331749,0.325091,0.062040,2.26215,4.33943,-0.224999,0.233586,3.381280,1.90299,0.067874,-0.051268,0.006135,2.60444,0.103441,0.067638,4.75362,1.85552,-0.181834,0.008359,3.16634,0.011850,0.022292,0.069320,0.117109,0.315276,24.480700,1.67227,-0.409067,4.95475,0.092358,2.60318,1.954690,0.005896,3.289340,2.564530,0.817706,0.025997,4.61749,1.575540,0.066105,0.681634,0.025247,0.183500,0.110038,2.74612,0.835586,0.188196,4.960640,0.136087,-0.008493,-0.015264,-0.010841,0.064584,0.102548,0.093611,0.964089,0.630422,4.307340,0.091289,-0.036360,3.61767,3.103240,0.000657,0.051302,1.92462,0.123294,-0.022671,1.548120,-0.010397,0.058330,3.66131,-0.118386,2.35739,-0.009112,0.178701,4.097350,3.53289,0.005244,0.121381,0.109968,0.135838,3.46017,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0.0,213.345703,2.133457,8.025316,76.533600,-0.471318,76.830978
2,2,0.036330,1.49747,233.546005,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,0.056995,0.328684,2.968810,0.105244,2.06949,5.30986,1.354790,-0.262018,1.379080,1.48091,0.020542,-0.008806,0.109348,1.68365,0.038180,0.123716,1.11248,3.57166,0.120601,0.082069,2.23352,0.002270,0.045182,0.014405,0.011599,-0.502849,33.738201,1.41750,1.071350,3.22296,2.122030,3.08216,0.637555,-0.006822,-0.390943,17.345699,3.700430,-0.033600,1.57824,0.051978,-0.002004,2.690960,0.018367,-0.030468,0.111409,2.18747,-0.325000,-0.019944,3.455030,0.068105,-0.009812,-0.010627,0.027571,-0.007121,-0.048914,-0.002574,1.865090,2.404170,0.411741,0.057749,0.525174,2.16879,0.828297,0.089848,0.093744,4.94901,-0.010978,0.076671,0.266784,0.038691,0.382731,3.84760,-0.121482,3.74084,0.147098,-0.016565,0.614651,2.12584,0.078828,0.979808,0.026758,0.117310,4.88300,0.085222,0.032396,0.116092,-0.001689,-0.520069,2.14112,0.124464,0.148209,0.0,382.921692,3.829217,23.531256,233.546005,-0.520069,94.408302


### Spliting the dataset back into train and test 

In [25]:
#Spliting train and test

train=dataset.iloc[:len(df_train)]
test=dataset.iloc[len(df_train):]
test.drop(labels=["target","id"],axis = 1,inplace=True)
train.drop(labels=["id"],axis = 1,inplace=True)
#train.drop(columns='id', inplace=True)
train.shape,test.shape

((600000, 107), (540000, 106))

#### Considering only 300000 random samples to simplify the computation

Since the first run with all the samples takes too long in k-fold, i have taken only 30k random samples from the train data set to train and k fold validation

In [26]:
# seed = 322
# df_train_30k = train.sample(n=30000, random_state=seed)


# np.random.seed(seed) 
# features_choice = np.random.choice(
#     df_train_30k.keys()[1:-1], size=12, replace=False
# )

# df_train_30k.head()

In [27]:
# Creating X_train and Y_train data
X_train=train.drop(columns=['target'])
y_train=train['target']

In [28]:
X_train.columns

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f96', 'f97', 'f98', 'f99', 'sum', 'mean', 'std', 'max', 'min', 'kurt'],
      dtype='object', length=106)

In [29]:
test.columns

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9',
       ...
       'f96', 'f97', 'f98', 'f99', 'sum', 'mean', 'std', 'max', 'min', 'kurt'],
      dtype='object', length=106)

## Base Line Model Creation

Below code is used for cross validation and model selection , and selected LinearDiscriminantAnalysis() , since it gives best score.
And created a model 

To make understanding simple comment out this process and directly used LinearDiscriminantAnalysis()  model for model creation

In [30]:
# # Cross validate model with Kfold stratified cross val
# kfold = StratifiedKFold(n_splits=10)

# # Modeling step Test differents algorithms 
# random_state = 2
# classifiers = []
# classifiers.append(SVC(random_state=random_state))
# classifiers.append(DecisionTreeClassifier(random_state=random_state))
# classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
# classifiers.append(RandomForestClassifier(random_state=random_state))
# classifiers.append(ExtraTreesClassifier(random_state=random_state))
# classifiers.append(GradientBoostingClassifier(random_state=random_state))
# classifiers.append(MLPClassifier(random_state=random_state))
# classifiers.append(KNeighborsClassifier())
# classifiers.append(LogisticRegression(random_state = random_state),solver='liblinear')
# classifiers.append(LinearDiscriminantAnalysis())


# # result into a list 

# cv_results = []
# for classifier in classifiers :
#     cv_results.append(cross_val_score(classifier, X_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))
    
# # mean and std on a list
# cv_means = []
# cv_std = []
# for cv_result in cv_results:
#     cv_means.append(cv_result.mean())
#     cv_std.append(cv_result.std())
    


### Model Creation

In [31]:
clf_lda= LinearDiscriminantAnalysis()#n_estimator is the only main hp we optimize in RF,
#In practice, adjusting only one of these (e.g. max_depth) is enough to reduce overfitting
clf_lda.fit(X_train, y_train)

clf_lda.score(X_train,y_train)

y_test_predict= clf_lda.predict(test)  # this is just to predict like a normal binary classification
y_test_predict_proba_lda= clf_lda.predict_proba(test)[:,1]

below code is to submit the probability for LDA model only

In [32]:
# submission = pd.DataFrame({'id': df_test['id'], 'target': y_test_predict_proba_lda})
# submission.to_csv('submission.csv', index=False)
# submission.head(3)

### Ensemble model with logistic and LDA

In [33]:
random_state = 2
clf_log = LogisticRegression(random_state = random_state,solver='liblinear')
clf_log.fit(X_train,y_train)

clf_log.score(X_train,y_train)

#y_test_predict= clf_log.predict(test) 
y_test_predict_proba_log= clf_log.predict_proba(test)[:,1]

### Ensemble model creation using Log and LDA

In [34]:
ensemble = pd.DataFrame({'id': df_test['id'], 'target_log': y_test_predict_proba_log,'target_lda': y_test_predict_proba_lda})
targets=['target_lda','target_log']
ensemble.head(2)

,id,target_log,target_lda
0,600000,0.677798,0.682093
1,600001,0.620444,0.619702


### Submission

In [35]:
#Submission
submission = pd.DataFrame({'id': df_test['id'], 'target': ensemble[targets].mean(axis=1)})
submission.to_csv('submission.csv', index=False)